In [1]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go

# Load the dataset
df = pd.read_csv('customer_segmentation_clusters.csv')

# Specify the valid customer regions
valid_customer_regions = [4660, 8670, 2360, 2440, 4140, 2490, 8370, 8550]

# Initialize the Dash app
app = dash.Dash(__name__)

# Define dark and light mode layouts
dark_mode_layout = go.Layout(
    paper_bgcolor='rgb(17, 17, 17)',
    plot_bgcolor='rgb(17, 17, 17)',
    font=dict(color='white'),
    title=dict(font=dict(color='white')),
    xaxis=dict(title=dict(font=dict(color='white')), tickfont=dict(color='white')),
    yaxis=dict(title=dict(font=dict(color='white')), tickfont=dict(color='white'))
)

light_mode_layout = go.Layout(
    paper_bgcolor='white',
    plot_bgcolor='white',
    font=dict(color='black'),
    title=dict(font=dict(color='black')),
    xaxis=dict(title=dict(font=dict(color='black')), tickfont=dict(color='black')),
    yaxis=dict(title=dict(font=dict(color='black')), tickfont=dict(color='black'))
)

# Define the layout of the app
app.layout = html.Div([
    html.H1(id="main-title", children="Customer Segmentation Dashboard"),
    
    # Theme toggle switch
    html.Div([
        html.Label(id="theme-label", children="Select Theme"),
        dcc.RadioItems(
            id='theme-switch',
            options=[
                {'label': 'Light Mode', 'value': 'light'},
                {'label': 'Dark Mode', 'value': 'dark'},
            ],
            value='light',
            labelStyle={'display': 'inline-block', 'margin-right': '10px'}
        )
    ], style={'padding': '10px', 'backgroundColor': 'lightgray'}),
    
    # Cluster Exploration Section
    html.Div([
        html.H2(id="cluster-title", children="Cluster Exploration"),
        
        # Dropdown for cluster selection
        html.Label(id="cluster-label", children="Select Cluster"),
        dcc.Dropdown(
            id='cluster-dropdown',
            options=[{'label': 'All', 'value': 'All'}] + [{'label': str(cluster), 'value': str(cluster)} for cluster in df['final_cluster'].unique()],
            value='All'
        ),
        
        # Slider for age range selection
        html.Label(id="age-range-label", children="Select Age Range"),
        dcc.RangeSlider(
            id='age-slider',
            min=df['customer_age'].min(),
            max=df['customer_age'].max(),
            value=[df['customer_age'].min(), df['customer_age'].max()],
            marks={i: str(i) for i in range(df['customer_age'].min(), df['customer_age'].max() + 1, 5)},
            tooltip={"placement": "bottom", "always_visible": True}
        ),
        
        # Bar chart for customer region distribution
        dcc.Graph(id='region-bar-chart')
    ]),
    
    # Visualization Tools Section
    html.Div([
        html.H2(id="visualization-title", children="Visualization Tools"),
        
        # Heatmap for demographics
        dcc.Graph(id='demographics-heatmap'),
        
        # Tree map for preferred cuisines
        dcc.Graph(id='cuisines-tree-map'),

        # Line chart for order trends
        dcc.Graph(id='order-trend-line-chart'),

        # Box plot for customer spending
        dcc.Graph(id='spending-box-plot'),

        # Histogram for order frequency
        dcc.Graph(id='order-frequency-histogram'),

        # Pie chart for payment method distribution
        dcc.Graph(id='payment-method-pie-chart'),

        # Scatter plot for customer spending vs age
        dcc.Graph(id='spending-age-scatter-plot'),

        # Heatmap for active days
        dcc.Graph(id='active-days-heatmap'),

        # Bar chart for promotion impact
        dcc.Graph(id='promotion-impact-bar-chart')
    ])
])

# Define the callback to update the bar chart
@app.callback(
    Output('region-bar-chart', 'figure'),
    [Input('cluster-dropdown', 'value'),
     Input('age-slider', 'value'),
     Input('theme-switch', 'value')]
)
def update_bar_chart(selected_cluster, age_range, theme):
    # Filter the dataframe based on the selected cluster and age range
    if selected_cluster == 'All' or selected_cluster is None:
        filtered_df = df[(df['customer_age'] >= age_range[0]) & (df['customer_age'] <= age_range[1])]
    else:
        selected_cluster = int(selected_cluster)
        filtered_df = df[(df['final_cluster'] == selected_cluster) & (df['customer_age'] >= age_range[0]) & (df['customer_age'] <= age_range[1])]

    # Filter the DataFrame to only include valid customer regions
    filtered_df = filtered_df[filtered_df['customer_region'].isin(valid_customer_regions)]
    
    # Convert customer_region and final_cluster to string for categorical handling
    filtered_df['customer_region'] = filtered_df['customer_region'].astype(str)
    filtered_df['final_cluster'] = filtered_df['final_cluster'].astype(str)
    
    # Create the bar chart for customer_region
    region_counts = filtered_df['customer_region'].value_counts().reset_index()
    region_counts.columns = ['customer_region', 'count']
    fig = px.bar(region_counts, x='customer_region', y='count', color='customer_region', title='Distribution of Customer Region', labels={'customer_region': 'Customer Region', 'count': 'Count'})
    fig.update_layout(dark_mode_layout if theme == 'dark' else light_mode_layout)
    return fig

# Define the callback to update the visualizations
@app.callback(
    [Output('main-title', 'style'),
     Output('theme-label', 'style'),
     Output('cluster-title', 'style'),
     Output('cluster-label', 'style'),
     Output('age-range-label', 'style'),
     Output('visualization-title', 'style'),
     Output('demographics-heatmap', 'figure'),
     Output('cuisines-tree-map', 'figure'),
     Output('order-trend-line-chart', 'figure'),
     Output('spending-box-plot', 'figure'),
     Output('order-frequency-histogram', 'figure'),
     Output('payment-method-pie-chart', 'figure'),
     Output('spending-age-scatter-plot', 'figure'),
     Output('active-days-heatmap', 'figure'),
     Output('promotion-impact-bar-chart', 'figure')],
    [Input('cluster-dropdown', 'value'),
     Input('age-slider', 'value'),
     Input('theme-switch', 'value')]
)
def update_visualizations(selected_cluster, age_range, theme):
    # Filter the dataframe based on the selected cluster and age range
    if selected_cluster == 'All' or selected_cluster is None:
        filtered_df = df[(df['customer_age'] >= age_range[0]) & (df['customer_age'] <= age_range[1])]
    else:
        selected_cluster = int(selected_cluster)
        filtered_df = df[(df['final_cluster'] == selected_cluster) & (df['customer_age'] >= age_range[0]) & (df['customer_age'] <= age_range[1])]
    
    # Filter the DataFrame to only include valid customer regions
    filtered_df = filtered_df[filtered_df['customer_region'].isin(valid_customer_regions)]
    
    # Convert customer_region and final_cluster to string for categorical handling
    filtered_df['customer_region'] = filtered_df['customer_region'].astype(str)
    filtered_df['final_cluster'] = filtered_df['final_cluster'].astype(str)
    
    # Define the title and label styles based on the selected theme
    title_style = {'color': 'white'} if theme == 'dark' else {'color': 'black'}
    label_style = {'color': 'white'} if theme == 'dark' else {'color': 'black'}
    
    # Create the heatmap for demographics
    demographics_heatmap = px.density_heatmap(
        filtered_df, 
        x='customer_region', 
        y='customer_age', 
        title='Heatmap of Customer Region vs Age',
        labels={'customer_region': 'Customer Region', 'customer_age': 'Customer Age'}
    )
    demographics_heatmap.update_layout(dark_mode_layout if theme == 'dark' else light_mode_layout)
    
    # Create the tree map for preferred cuisines
    cuisines_columns = [col for col in df.columns if col.startswith('CUI_') and not col.endswith('_proportion')]
    cuisines_data = filtered_df[cuisines_columns].sum().reset_index()
    cuisines_data.columns = ['Cuisine', 'Count']
    cuisines_data['Cuisine'] = cuisines_data['Cuisine'].str.replace('CUI_', '')

    cuisines_tree_map = px.treemap(
        cuisines_data, 
        path=['Cuisine'], 
        values='Count', 
        title='Preferred Cuisines'
    )
    cuisines_tree_map.update_layout(dark_mode_layout if theme == 'dark' else light_mode_layout)

    # Create the line chart for order trends
    order_trend_line_chart = px.line(
        filtered_df, 
        x='first_order', 
        y='total_orders', 
        title='Order Trends Over Time',
        labels={'first_order': 'First Order Time', 'total_orders': 'Total Orders'}
    )
    order_trend_line_chart.update_layout(dark_mode_layout if theme == 'dark' else light_mode_layout)

    # Create the box plot for customer spending
    spending_box_plot = px.box(
        filtered_df, 
        x='final_cluster', 
        y='total_spent', 
        title='Customer Spending Across Clusters',
        labels={'final_cluster': 'Cluster', 'total_spent': 'Total Spent'}
    )
    spending_box_plot.update_layout(dark_mode_layout if theme == 'dark' else light_mode_layout)

    # Create the histogram for order frequency
    order_frequency_histogram = px.histogram(
        filtered_df, 
        x='total_orders', 
        title='Order Frequency Distribution',
        labels={'total_orders': 'Total Orders'}
    )
    order_frequency_histogram.update_layout(dark_mode_layout if theme == 'dark' else light_mode_layout)

    # Create the pie chart for payment method distribution
    payment_method_pie_chart = px.pie(
        filtered_df, 
        names='payment_method', 
        title='Payment Method Distribution'
    )
    payment_method_pie_chart.update_layout(dark_mode_layout if theme == 'dark' else light_mode_layout)

    # Create the scatter plot for customer spending vs age
    spending_age_scatter_plot = px.scatter(
        filtered_df, 
        x='customer_age', 
        y='total_spent', 
        title='Customer Spending vs Age',
        labels={'customer_age': 'Customer Age', 'total_spent': 'Total Spent'}
    )
    spending_age_scatter_plot.update_layout(dark_mode_layout if theme == 'dark' else light_mode_layout)

    # Create the heatmap for active days
    active_days_heatmap = px.density_heatmap(
        filtered_df, 
        x='active_days', 
        y='final_cluster', 
        title='Heatmap of Active Days Across Clusters',
        labels={'active_days': 'Active Days', 'final_cluster': 'Cluster'}
    )
    active_days_heatmap.update_layout(dark_mode_layout if theme == 'dark' else light_mode_layout)

    # Create the bar chart for promotion impact
    promotion_impact_bar_chart = px.bar(
        filtered_df, 
        x='final_cluster', 
        y='had_promotion', 
        title='Promotion Impact Across Clusters',
        labels={'final_cluster': 'Cluster', 'had_promotion': 'Had Promotion'}
    )
    promotion_impact_bar_chart.update_layout(dark_mode_layout if theme == 'dark' else light_mode_layout)
    
    return (title_style, title_style, title_style, label_style, label_style, title_style, 
            demographics_heatmap, cuisines_tree_map, order_trend_line_chart, 
            spending_box_plot, order_frequency_histogram, payment_method_pie_chart, 
            spending_age_scatter_plot, active_days_heatmap, promotion_impact_bar_chart)

# Run the Dash app on a different port
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)